In [1]:
import sys, os, importlib.util
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd

from scipy.constants import lambda2nu, nu2lambda, c
from scipy.interpolate import interp1d
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset

In [2]:
from IPython.core.display import HTML
from IPython.core.pylabtools import figsize
from IPython.display import display, Math

In [3]:
lumapiFile = "C:\\Program Files\\Lumerical\\v221\\api\\python\\lumapi.py"
spec = importlib.util.spec_from_file_location("lumapi", lumapiFile)
lumapi = importlib.util.module_from_spec(spec)
spec.loader.exec_module(lumapi)

modeApi = lumapi.MODE(hide = False)

In [4]:
um = 1e-6
nm = 1e-9
valor_lambda = 1550*nm

In [5]:
ng = 4.3458796
FSR = [1*nm, 10*nm, 20*nm]

L1 = np.zeros(len(FSR))
L2 = np.zeros(len(FSR))

for i in range (len(FSR)):
    d_L = valor_lambda**2/(ng*FSR[i])
    L1[i] = 20*um
    L2[i] = L1[i] + d_L

    print(f'\u0394L = {d_L/um:.2f}\u03BCm\nComprimento maior = {L2[i]/um:.2f}\u03BCm\nComprimento menor = {L1[i]/um:.2f}\u03BCm')

ΔL = 552.82μm
Comprimento maior = 572.82μm
Comprimento menor = 20.00μm
ΔL = 55.28μm
Comprimento maior = 75.28μm
Comprimento menor = 20.00μm
ΔL = 27.64μm
Comprimento maior = 47.64μm
Comprimento menor = 20.00μm


In [6]:
sub_material = 'SiO2 (Glass) - Palik'

x_span_waveguide = np.min(L1)
y_span_waveguide = 0.450*um
z_span_waveguide = 0.22*um
x_waveguide = 0.0*um
y_waveguide = 0.0*um
z_waveguide = 0.0*um
waveguide_material = 'Si (Silicon) - Palik'

In [7]:
modeApi.switchtolayout()

modeApi.select('GDS_LAYER_35:4')
modeApi.delete()

modeApi.addstructuregroup()
dev_layer = 'GDS_LAYER_35:4'
modeApi.set('name', dev_layer)
modeApi.set('x', x_waveguide)
modeApi.set('y', y_waveguide)

In [8]:
modeApi.switchtolayout()

modeApi.select(f'{dev_layer}::waveguide')
modeApi.delete()

modeApi.addrect()
modeApi.addtogroup(dev_layer)
modeApi.set('name', 'waveguide')
modeApi.set('material', waveguide_material)
modeApi.set('y', -y_waveguide)
modeApi.set('x', x_waveguide)
modeApi.set('z', z_waveguide)
modeApi.set('y span', y_span_waveguide)
modeApi.set('z span', z_span_waveguide)

In [9]:
x_fde = 0.0*um
y_fde = 0.0*um
z_fde = 0.0*um
x_span_fde = 0*um
y_span_fde = 2.5*um
z_span_fde = 1*um

mesh_cells = 100
mesh_multiplier = 5

wavelength = 1550*nm
start_wavelength = 1500*nm
stop_wavelength = 1600*nm

modes = 4

number_points = 100

name = 'Mode_waveguide'

In [10]:
modeApi.switchtolayout()

modeApi.select('FDE')
modeApi.delete()

modeApi.addfde()
modeApi.set('solver type', '2D X normal')
modeApi.set('background material', sub_material)

modeApi.set('x', x_fde)
modeApi.set('y', y_fde)
modeApi.set('z', z_fde)
modeApi.set('z span', z_span_fde)
modeApi.set('y span', y_span_fde)

modeApi.set('define z mesh by', 'number of mesh cells')
modeApi.set('mesh cells z', mesh_cells)
modeApi.set('define y mesh by', 'number of mesh cells')
modeApi.set('mesh cells y', mesh_cells)

modeApi.set('wavelength', wavelength)

modeApi.set('number of trial modes', modes)

modeApi.set('fit materials with multi-coefficient model', True)
modeApi.set('wavelength start', start_wavelength)
modeApi.set('wavelength stop', stop_wavelength)

modeApi.set('z min bc', 'Metal')
modeApi.set('z max bc', 'Metal')
modeApi.set('y min bc', 'Metal')
modeApi.set('y max bc', 'Metal')

In [11]:
modeApi.switchtolayout()

modeApi.select('mesh')
modeApi.delete()

modeApi.addmesh()
modeApi.set('name','mesh_box')
modeApi.set('based on a structure', True)
modeApi.set('structure', 'waveguide')

modeApi.set('set mesh multiplier', True)
modeApi.set("x mesh multiplier", mesh_multiplier)
modeApi.set("y mesh multiplier", mesh_multiplier)
modeApi.set("z mesh multiplier", mesh_multiplier)

In [12]:
modeApi.save(name)

modeApi.findmodes()

4.0

In [13]:
modeApi.selectmode(1)

modeApi.setanalysis('track selected mode', 1)
modeApi.setanalysis('stop wavelength', stop_wavelength)
modeApi.setanalysis('number of test modes', modes)
modeApi.setanalysis('number of points', number_points)
modeApi.setanalysis("detailed dispersion calculation",1)
modeApi.setanalysis("store mode profiles while tracking",1)


In [14]:

modeApi.frequencysweep()